In [ ]:
library(haven)
options(scipen = 4)

In [ ]:
#load Ed Data
meapdata<-read_dta("MEAP93.dta")
head(meapdata)

In [ ]:
#regress graduation rates on enrollment, total compensation, number of staff, and eligibility for free lunch
reg1<-lm(gradrate~enroll + totcomp + staff + lnchprg, data = meapdata)
summary(reg1)

In [ ]:
str(summary(reg1))

In [ ]:
summary(reg1)$coefficients

In [ ]:
# Confidence intervals
# Find the critical value
c = qt(p=0.05/2, df=403,lower.tail=F)
# Calculate intervals
lci=round(summary(reg1)$coefficients[2,1] - c* summary(reg1)$coefficients[2,2],5)
rci=round(summary(reg1)$coefficients[2,1] + c* summary(reg1)$coefficients[2,2],5)
# Print result
print(paste0("The estimated 95% confidence interval for beta_2 is [",lci,",",rci,"]"))

In [ ]:
# What does the confidence interval imply about the economic significance of eligibility for school lunch?

# Back to lecture

In [ ]:
#load college dataset
twoyeardata<- read_dta("twoyear.dta")
head(twoyeardata)

In [ ]:
#generate regression model
reg2 <- lm(lwage ~ jc + univ + exper, data = twoyeardata)
summary(reg2)
# Can we test beta_1-beta_2=0?

In [ ]:
# Back to lecture

In [ ]:
#test for equality of coefficients: rewrite the model
twoyeardata$allcoll = twoyeardata$jc+twoyeardata$univ
reg3 <- lm(lwage ~ jc + allcoll + exper, data = twoyeardata)
summary(reg3)
# What is the result of our hypothesis test?
# How do I interpret the coefficients on jc and allcoll?

In [ ]:
# Notice relationships between coefficients
summary(reg3)$coefficients[3,1] # allcoll coeff
summary(reg2)$coefficients[3,1] # univ coeff
summary(reg3)$coefficients[3,1]+summary(reg3)$coefficients[2,1] # allcoll + jc coeff
summary(reg2)$coefficients[2,1] # jc coeff

In [ ]:
#test for equality of coefficients alternative: need "car" package
library(car)
#command linearHypothesis takes the regression object, and then states the model
linearHypothesis(reg2, "jc = univ")
# Notice same p-value
round(linearHypothesis(reg2, "jc = univ")[2,6],4)==round(summary(reg3)$coefficients[2,4],4)

In [ ]:
# Back to lecture

In [ ]:
#read in birthweight data
bweight <- read_dta("BWGHT.DTA")
head(bweight)

In [ ]:
#regress birthweight in ounces on daily maternal cigarette consumption and parity (birth order)
reg4<-lm(bwght~cigs + parity + faminc, data = bweight)
summary(reg4)
# How do we interpret beta_1? sign, size, significance

In [ ]:
# Back to lecture

In [ ]:
#Add SES variables
reg5 <- lm(bwght~cigs + parity + faminc + motheduc + fatheduc, data = bweight)
summary(reg5)

In [ ]:
# For consistency, need the same sample in both models
# Only keep observations where all variables are not missing for F calculation
bweight<- subset(bweight, is.na(fatheduc) == F & is.na(motheduc)== F)
#regress birthweight on cigarette consumption,  birth order, and family income
reg6<- lm(bwght ~ cigs + parity + faminc, data = bweight)
summary(reg6)

In [ ]:
#include family education controls
reg7 <- lm(bwght ~ cigs + parity + faminc + motheduc + fatheduc, data = bweight)
summary(reg7)

In [ ]:
#calculate sums of squared residuals
#unrestricted model
SSRU<-sum(resid(reg7)^2)
SSRU
#restricted model
SSRR<-sum(resid(reg6)^2)
SSRR
#identify n
n <- nobs(reg7)
n

In [ ]:
#calculate F statistic
# number of restrictions/numerator df
q=2
F1<- round(((SSRR-SSRU)/q)/(SSRU/(n-5-1)),4)
F1

In [ ]:
# Get the critical value
# syntax: qf(p, df1, df2. lower.tail=TRUE)
F_crit=round(qf(0.05,q,n-5-1,lower.tail=FALSE),4)
print(paste0("Is ",F1," > ",F_crit,"? ",F1>F_crit))

In [ ]:
# Back to lecture

In [ ]:
#R-squared calculation
#Pull unrestricted R-squared from regression output
Rsq_u <-summary(reg7)$r.squared
Rsq_u
#Pull restricted R-squared from regression output
Rsq_r <-summary(reg6)$r.squared
Rsq_r
F2 <- round(((Rsq_u-Rsq_r)/q)/((1-Rsq_u)/(n-5-1)),4)
F2
F2==F1

In [ ]:
# Use linearHypothesis test 
linearHypothesis(reg7,c("motheduc = 0", "fatheduc = 0"))
F3<- round(linearHypothesis(reg7,c("motheduc = 0", "fatheduc = 0"))[2,5],4)
F3==F1
# Advantage that this also gives you a p-value

In [ ]:
# But can also get p-value directly from F distribution
# syntax pf(F-stat, q, denominator df, lower.tail=TRUE)
pf(F3, q, n-5-1, lower.tail = FALSE)

In [ ]:
# Overall F-statistic
q=5
F_overall=round(((Rsq_u)/q)/((1-Rsq_u)/(n-q-1)),4)
F_overall
summary(reg7)

In [ ]:
str(summary(reg7))

In [ ]:
summary(reg7)$fstatistic
round(summary(reg7)$fstatistic[1],4)==F_overall